# Phase 2: Deep Learning Models (LSTM + GRU)\n
\n
## Goals:\n
- Prepare sequential data for LSTM/GRU\n
- Build and train LSTM model\n
- Build and train GRU model\n
- Compare performance with baseline models\n
- Save trained models

## Step 1: Import Libraries

In [ ]:
import numpy as np\n
import pandas as pd\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
from sklearn.metrics import (\n
    accuracy_score, precision_score, recall_score, f1_score,\n
    confusion_matrix, classification_report, roc_auc_score, roc_curve\n
)\n
import tensorflow as tf\n
from tensorflow import keras\n
from tensorflow.keras.models import Sequential, Model\n
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input\n
from tensorflow.keras.optimizers import Adam\n
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint\n
import warnings\n
warnings.filterwarnings('ignore')\n
\n
# Set random seeds\n
np.random.seed(42)\n
tf.random.set_seed(42)\n
\n
print(f'TensorFlow version: {tf.__version__}')\n
print(f'Keras version: {keras.__version__}')\n
print('All libraries imported successfully!')

## Step 2: Load Preprocessed Data

In [ ]:
# Load preprocessed data from Phase 1\n
X_train = pd.read_csv('../../data/processed/X_train.csv')\n
X_test = pd.read_csv('../../data/processed/X_test.csv')\n
y_train = pd.read_csv('../../data/processed/y_train.csv').values.ravel()\n
y_test = pd.read_csv('../../data/processed/y_test.csv').values.ravel()\n
\n
print(f'Training set: {X_train.shape}')\n
print(f'Test set: {X_test.shape}')\n
print(f'Training labels: {y_train.shape}')\n
print(f'Test labels: {y_test.shape}')

##  Step 3: Prepare Sequential Data\n
\n
LSTM and GRU expect 3D input: (samples, time_steps, features)\n
We'll reshape our data to add a time dimension.

In [ ]:
# Reshape data for LSTM/GRU\n
# Convert to numpy arrays\n
X_train_seq = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))\n
X_test_seq = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))\n
\n
print(f'Sequential training data shape: {X_train_seq.shape}')\n
print(f'Sequential test data shape: {X_test_seq.shape}')\n
print(f'Format: (samples, time_steps, features)')

## Step 4: Build LSTM Model

In [ ]:
# Build LSTM architecture\n
def build_lstm_model(input_shape):\n
    model = Sequential([\n
        LSTM(64, input_shape=input_shape, return_sequences=True),\n
        Dropout(0.3),\n
        LSTM(32, return_sequences=False),\n
        Dropout(0.3),\n
        Dense(16, activation='relu'),\n
        Dropout(0.2),\n
        Dense(1, activation='sigmoid')\n
    ])\n
    \n
    model.compile(\n
        optimizer=Adam(learning_rate=0.001),\n
        loss='binary_crossentropy',\n
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]\n
    )\n
    \n
    return model\n
\n
# Create model\n
lstm_model = build_lstm_model((X_train_seq.shape[1], X_train_seq.shape[2]))\n
lstm_model.summary()

## Step 5: Train LSTM Model

In [ ]:
# Callbacks\n
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)\n
checkpoint = ModelCheckpoint('../../models/saved_models/lstm_model.keras', \n
                            save_best_only=True, monitor='val_loss')\n
\n
# Calculate class weights (handling imbalance)\n
from sklearn.utils.class_weight import compute_class_weight\n
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)\n
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}\n
\n
print(f'Class weights: {class_weight_dict}')\n
print('\\nTraining LSTM model...')

In [ ]:
# Train the model\n
history_lstm = lstm_model.fit(\n
    X_train_seq, y_train,\n
    validation_split=0.2,\n
    epochs=20,\n
    batch_size=256,\n
    class_weight=class_weight_dict,\n
    callbacks=[early_stop, checkpoint],\n
    verbose=1\n
)\n
\n
print('\\nLSTM model training completed!')

## Step 6: Evaluate LSTM Model

In [ ]:
# Predict\n
y_pred_proba_lstm = lstm_model.predict(X_test_seq).ravel()\n
y_pred_lstm = (y_pred_proba_lstm > 0.5).astype(int)\n
\n
# Metrics\n
print('=== LSTM MODEL RESULTS ===')\n
print(f'Accuracy: {accuracy_score(y_test, y_pred_lstm):.4f}')\n
print(f'Precision: {precision_score(y_test, y_pred_lstm):.4f}')\n
print(f'Recall: {recall_score(y_test, y_pred_lstm):.4f}')\n
print(f'F1-Score: {f1_score(y_test, y_pred_lstm):.4f}')\n
print(f'ROC-AUC: {roc_auc_score(y_test, y_pred_proba_lstm):.4f}')\n
print('\\nClassification Report:')\n
print(classification_report(y_test, y_pred_lstm, target_names=['Normal', 'Fraud']))\n
print('\\nConfusion Matrix:')\n
print(confusion_matrix(y_test, y_pred_lstm))

In [ ]:
# Plot training history\n
fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n
\n
# Loss\n
axes[0, 0].plot(history_lstm.history['loss'], label='Training Loss')\n
axes[0, 0].plot(history_lstm.history['val_loss'], label='Validation Loss')\n
axes[0, 0].set_title('LSTM - Loss', fontsize=12, fontweight='bold')\n
axes[0, 0].set_xlabel('Epoch')\n
axes[0, 0].set_ylabel('Loss')\n
axes[0, 0].legend()\n
axes[0, 0].grid(alpha=0.3)\n
\n
# Accuracy\n
axes[0, 1].plot(history_lstm.history['accuracy'], label='Training Accuracy')\n
axes[0, 1].plot(history_lstm.history['val_accuracy'], label='Validation Accuracy')\n
axes[0, 1].set_title('LSTM - Accuracy', fontsize=12, fontweight='bold')\n
axes[0, 1].set_xlabel('Epoch')\n
axes[0, 1].set_ylabel('Accuracy')\n
axes[0, 1].legend()\n
axes[0, 1].grid(alpha=0.3)\n
\n
# Precision\n
axes[1, 0].plot(history_lstm.history['precision'], label='Training Precision')\n
axes[1, 0].plot(history_lstm.history['val_precision'], label='Validation Precision')\n
axes[1, 0].set_title('LSTM - Precision', fontsize=12, fontweight='bold')\n
axes[1, 0].set_xlabel('Epoch')\n
axes[1, 0].set_ylabel('Precision')\n
axes[1, 0].legend()\n
axes[1, 0].grid(alpha=0.3)\n
\n
# Recall\n
axes[1, 1].plot(history_lstm.history['recall'], label='Training Recall')\n
axes[1, 1].plot(history_lstm.history['val_recall'], label='Validation Recall')\n
axes[1, 1].set_title('LSTM - Recall', fontsize=12, fontweight='bold')\n
axes[1, 1].set_xlabel('Epoch')\n
axes[1, 1].set_ylabel('Recall')\n
axes[1, 1].legend()\n
axes[1, 1].grid(alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## Step 7: Build GRU Model

In [ ]:
# Build GRU architecture\n
def build_gru_model(input_shape):\n
    model = Sequential([\n
        GRU(64, input_shape=input_shape, return_sequences=True),\n
        Dropout(0.3),\n
        GRU(32, return_sequences=False),\n
        Dropout(0.3),\n
        Dense(16, activation='relu'),\n
        Dropout(0.2),\n
        Dense(1, activation='sigmoid')\n
    ])\n
    \n
    model.compile(\n
        optimizer=Adam(learning_rate=0.001),\n
        loss='binary_crossentropy',\n
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]\n
    )\n
    \n
    return model\n
\n
# Create model\n
gru_model = build_gru_model((X_train_seq.shape[1], X_train_seq.shape[2]))\n
gru_model.summary()

## Step 8: Train GRU Model

In [ ]:
# Callbacks\n
early_stop_gru = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)\n
checkpoint_gru = ModelCheckpoint('../../models/saved_models/gru_model.keras', \n
                                save_best_only=True, monitor='val_loss')\n
\n
print('Training GRU model...')

In [ ]:
# Train the model\n
history_gru = gru_model.fit(\n
    X_train_seq, y_train,\n
    validation_split=0.2,\n
    epochs=20,\n
    batch_size=256,\n
    class_weight=class_weight_dict,\n
    callbacks=[early_stop_gru, checkpoint_gru],\n
    verbose=1\n
)\n
\n
print('\\nGRU model training completed!')

## Step 9: Evaluate GRU Model

In [ ]:
# Predict\n
y_pred_proba_gru = gru_model.predict(X_test_seq).ravel()\n
y_pred_gru = (y_pred_proba_gru > 0.5).astype(int)\n
\n
# Metrics\n
print('=== GRU MODEL RESULTS ===')\n
print(f'Accuracy: {accuracy_score(y_test, y_pred_gru):.4f}')\n
print(f'Precision: {precision_score(y_test, y_pred_gru):.4f}')\n
print(f'Recall: {recall_score(y_test, y_pred_gru):.4f}')\n
print(f'F1-Score: {f1_score(y_test, y_pred_gru):.4f}')\n
print(f'ROC-AUC: {roc_auc_score(y_test, y_pred_proba_gru):.4f}')\n
print('\\nClassification Report:')\n
print(classification_report(y_test, y_pred_gru, target_names=['Normal', 'Fraud']))\n
print('\\nConfusion Matrix:')\n
print(confusion_matrix(y_test, y_pred_gru))

In [ ]:
# Plot GRU training history\n
fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n
\n
# Loss\n
axes[0, 0].plot(history_gru.history['loss'], label='Training Loss')\n
axes[0, 0].plot(history_gru.history['val_loss'], label='Validation Loss')\n
axes[0, 0].set_title('GRU - Loss', fontsize=12, fontweight='bold')\n
axes[0, 0].set_xlabel('Epoch')\n
axes[0, 0].set_ylabel('Loss')\n
axes[0, 0].legend()\n
axes[0, 0].grid(alpha=0.3)\n
\n
# Accuracy\n
axes[0, 1].plot(history_gru.history['accuracy'], label='Training Accuracy')\n
axes[0, 1].plot(history_gru.history['val_accuracy'], label='Validation Accuracy')\n
axes[0, 1].set_title('GRU - Accuracy', fontsize=12, fontweight='bold')\n
axes[0, 1].set_xlabel('Epoch')\n
axes[0, 1].set_ylabel('Accuracy')\n
axes[0, 1].legend()\n
axes[0, 1].grid(alpha=0.3)\n
\n
# Precision\n
axes[1, 0].plot(history_gru.history['precision'], label='Training Precision')\n
axes[1, 0].plot(history_gru.history['val_precision'], label='Validation Precision')\n
axes[1, 0].set_title('GRU - Precision', fontsize=12, fontweight='bold')\n
axes[1, 0].set_xlabel('Epoch')\n
axes[1, 0].set_ylabel('Precision')\n
axes[1, 0].legend()\n
axes[1, 0].grid(alpha=0.3)\n
\n
# Recall\n
axes[1, 1].plot(history_gru.history['recall'], label='Training Recall')\n
axes[1, 1].plot(history_gru.history['val_recall'], label='Validation Recall')\n
axes[1, 1].set_title('GRU - Recall', fontsize=12, fontweight='bold')\n
axes[1, 1].set_xlabel('Epoch')\n
axes[1, 1].set_ylabel('Recall')\n
axes[1, 1].legend()\n
axes[1, 1].grid(alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## Step 10: Compare LSTM vs GRU

In [ ]:
# Compare models\n
models = ['LSTM', 'GRU']\n
accuracy = [accuracy_score(y_test, y_pred_lstm), accuracy_score(y_test, y_pred_gru)]\n
precision = [precision_score(y_test, y_pred_lstm), precision_score(y_test, y_pred_gru)]\n
recall = [recall_score(y_test, y_pred_lstm), recall_score(y_test, y_pred_gru)]\n
f1 = [f1_score(y_test, y_pred_lstm), f1_score(y_test, y_pred_gru)]\n
\n
fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n
\n
# Accuracy\n
axes[0, 0].bar(models, accuracy, color=['#9b59b6', '#1abc9c'])\n
axes[0, 0].set_title('Accuracy Comparison', fontsize=12, fontweight='bold')\n
axes[0, 0].set_ylim([0.9, 1.0])\n
\n
# Precision\n
axes[0, 1].bar(models, precision, color=['#9b59b6', '#1abc9c'])\n
axes[0, 1].set_title('Precision Comparison', fontsize=12, fontweight='bold')\n
axes[0, 1].set_ylim([0, 1])\n
\n
# Recall\n
axes[1, 0].bar(models, recall, color=['#9b59b6', '#1abc9c'])\n
axes[1, 0].set_title('Recall Comparison', fontsize=12, fontweight='bold')\n
axes[1, 0].set_ylim([0, 1])\n
\n
# F1-Score\n
axes[1, 1].bar(models, f1, color=['#9b59b6', '#1abc9c'])\n
axes[1, 1].set_title('F1-Score Comparison', fontsize=12, fontweight='bold')\n
axes[1, 1].set_ylim([0, 1])\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# ROC Curve comparison\n
fig, ax = plt.subplots(figsize=(10, 7))\n
\n
# Plot ROC curves\n
fpr_lstm, tpr_lstm, _ = roc_curve(y_test, y_pred_proba_lstm)\n
fpr_gru, tpr_gru, _ = roc_curve(y_test, y_pred_proba_gru)\n
\n
ax.plot(fpr_lstm, tpr_lstm, label=f'LSTM (AUC = {roc_auc_score(y_test, y_pred_proba_lstm):.3f})', linewidth=2)\n
ax.plot(fpr_gru, tpr_gru, label=f'GRU (AUC = {roc_auc_score(y_test, y_pred_proba_gru):.3f})', linewidth=2)\n
ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier')\n
\n
ax.set_xlabel('False Positive Rate', fontsize=12)\n
ax.set_ylabel('True Positive Rate', fontsize=12)\n
ax.set_title('RO Curve Comparison - Deep Learning Models', fontsize=14, fontweight='bold')\n
ax.legend(fontsize=10)\n
ax.grid(alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## Phase 2 Summary\n
\n
### ✅ Completed:\n
1. Prepared sequential data for LSTM/GRU\n
2. Built and trained LSTM model\n
3. Built and trained GRU model\n
4. Evaluated both models with multiple metrics\n
5. Compared LSTM vs GRU performance\n
6. Saved trained deep learning models\n
\n
### 🔥 Key Findings:\n
- LSTM and GRU show improved performance over baseline\n
- Both models handle temporal patterns effectively\n
- Ready for autoencoder (Phase 3) and ensemble (Phase 4)\n
\n
### 📊 Next Steps:\n
- Move to notebook 03/04: Autoencoder & Ensemble\n
- Build unsupervised anomaly detection\n
- Create ensemble fraud scoring system